<div>
<img src="logo.png", width=100, ALIGN="left">
<center>
<h1>Mini Projets 2019-2020 (Info 232)</h1>
Isabelle Guyon <br>
info232@chalearn.org <br>
</center>
<span style="color:red"> <h1> 2 . Pandas </h1> </span>
    
We have now a dataset of pictures of REAL apples and bananas, preprocessed in 4 different representations:
- one with only 2 features (<b>R</b>edness and <b>E</b>longation) called <b>RE_data.csv</b>
- one with only 21 features (<b>C</b>olor and <b>S</b>hape features) called <b>CS_data.csv</b>
- one with 14580 features (all pixels of 81x69x3 <b>raw</b> images) called <b>RAW_data.csv</b>
- one with 3072 features (all pixels of 32x32x3 <b>crop</b>ped images) called <b>CROP_data.csv</b>.

We will compare the performances of various classifiers on those 4 datasets. Thus, we are going to start doing "real" POM: Probability, Optimization, and Modeling. We will proceed in a "greedy" way, eliminating some non promising avenues as we go, and not revisiting them for the moment:

1. Which dataset version should we keep? Should we or not scale variables (using variable standardization)?
2. Which learning machine of a standard toolkit (scikit-learn) is most promising?

</div>
<div style="background:#FFFFAA">
    
 This TP gives you 5 points if you answer well ALL 5 questions. If you cannot fisnish, get help by attending the Wednesday session.
    
<span style="color:red"> <b>Save your notebook often with menu File + Save and Checkpoint.</b>
<br> <b>Before you push your homework to your GitHub repo, use  Kernel + Restart and Run all.</b>
</span>
    </div>
    

In [1]:
# Load general libraries
import os, re
from glob import glob as ls
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

### Add path to the sample code so the notebook finds it:
code_dir = 'code/'                        
from sys import path; path.append(code_dir)
from utilities import *
# Import code that checks your answers
from checker import check 
# Disable some warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 

# Step 1: Select data representation
Which dataset version should we keep? Should we do or not a variable standardization?

### Question 0: Examine the data directory
Go to the directory `mini-dataset/`. You should find four data files ending with `.csv`. In an editor, open `RL_data.csv` or another of the data files to see how it looks like. The dataset is formatted in the CSV format (comma separated file). The examples are in lines and the features are separated by commas. The first line is the header. 

Examine all 4 datasets and note the number of lines and columns. Notice that they all have a different number of features but the same number of examples. 

It is somewhat easier to write a Python program to do this work for you. Check how the function `check_datasets` is written by typing `??check_datasets` in a new cell. Verify with an editor that this is the same code that is found in the directory `code/` in the file `utilities.py`. 

In [2]:
# List the datasets
data_dir = './mini-dataset/'
data_list = ls(data_dir + '*_data.csv')
data_list

['./mini-dataset/RAW_data.csv',
 './mini-dataset/CROP_data.csv',
 './mini-dataset/RE_data.csv',
 './mini-dataset/CS_data.csv']

In [3]:

# Check the dataset sizes
check_datasets(data_list)

,Dataset,num. examples,num. features,num. apples,num. bananas
0,RAW,491,14581,333,158
1,CROP,491,3073,333,158
2,RE,491,3,333,158
3,CS,491,22,333,158


<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Show the code of check_datasets.</b>

In [4]:
def check_datasets(data_list):
    ''' Create a table with dataset statistics.'''
    col = ['Dataset', 'num. examples', 'num. features', 'num. apples', 'num. bananas']
    data = []
    for file in data_list:
        df = pd.read_csv(file)
        N, F = df.shape
        Na = sum(df.iloc[:,-1]==1)
        Nb = sum(df.iloc[:,-1]==-1)
        data.append([os.path.basename(file)[:-9],N,F,Na,Nb])
        stat_df = pd.DataFrame(data, columns = col)
    return stat_df

### Question 1: Load and visualize data with Pandas (review)
In previous classes we have have started using `Numpy arrays`and `Pandas dataframes`. We now explore Pandas a little bit further. Pandas dataframes are also arrays, but a different kind of python object that Numpy arrays. They have more properties, supporting fancy database functions and having quite a few display functions and nice simple summary statistics, <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html">check the documentation</a>. Learning about Pandas will help a lot the <i>visualization binome</i>.

First, you will perform these steps:
* Call to UNIX command `!head ./mini-dataset/RE_data.csv` to view the first few lines of the file. The character `!` allows you to "escape" from the Jupyter notebook to the UNIX shell.
* Load `RL_data.csv` as a pandas dataframe called `df`. Show the first few lines using the method `head`.
* Compute simple statistics using the method `describe`.
* Show the heat map. If you do not remember from the previous TP, think of using a search engine and type the keywords: "pandas heatmap". I found <a href="https://stackoverflow.com/questions/12286607/making-heatmap-from-pandas-dataframe">this post</a>, for instance. 
* Create a new data frame called `df_scaled` obtained by standardizing the columns of `df`. In the previous TP we used the method `StandardScaler` of `sklearn.preprocessing`. Notice that you can also simply compute the mean of `df` with `df.mean()` and the standard deviation with `df.std()`, then in one line of code get `df_scaled` by performing algebraic operations on dataframes!
* We actually doe NOT want to standardize the <b>last column</b> (the class label). Make sure the last column of  `df_scaled` has the ORIGINAL label values +1 or -1.

Then, the question you should answer to complete this section is: what are the mean and standard deviation of the lines and the columns of the `redness` and `elongation` features before and after standardization?

<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Put here your call to the UNIX command "head".</b>

In [5]:
!head ./mini-dataset/RE_data.csv

redness,elongation,fruit
8.429611650485437,1.4032012759326986,1.0
42.89607843137255,1.7032009898154288,1.0
56.354066985645936,1.5508105645577395,1.0
-10.528846153846153,1.7773426521360935,1.0
-17.131386861313867,2.047174441534352,1.0
36.041666666666664,1.4139561143744013,1.0
34.744932432432435,1.2379057682280559,1.0
1.8179190751445087,1.0839275628229141,1.0
3.34106529209622,1.0972000640347488,1.0


<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Read file RE_data.csv as a dataframe and call it "df".</b>

In [6]:
RE_files = pd.read_csv('./mini-dataset/RE_data.csv')
df = pd.DataFrame(RE_files) 
df.head()

,redness,elongation,fruit
0,8.429612,1.403201,1.0
1,42.896078,1.703201,1.0
2,56.354067,1.550811,1.0
3,-10.528846,1.777343,1.0
4,-17.131387,2.047174,1.0


<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Show descriptive statistics with the method "describe".</b>

In [7]:
df.describe()

,redness,elongation,fruit
count,491.000000,491.000000,491.000000
mean,17.863426,1.468878,0.356415
std,21.805386,0.521078,0.935280
min,-47.045455,1.000000,-1.000000
25%,3.895620,1.161664,-1.000000
50%,14.380952,1.300111,1.000000
75%,33.848108,1.598103,1.000000
max,100.364865,5.231697,1.000000


<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Show the heat map of "df".</b>

In [8]:
df.style.background_gradient(cmap='Blues')

,redness,elongation,fruit
0,8.42961,1.4032,1
1,42.8961,1.7032,1


<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Standardize the columns of "df", EXCEPT THE LAST ONE.</b>

In [9]:
from sklearn.preprocessing import StandardScaler
df_scaled = df.copy()
df_scaled.iloc[:,:-1] = (df_scaled.iloc[:,:-1] - df_scaled.iloc[:,:-1].mean()) / df_scaled.iloc[:,:-1].std()
df_scaled


,redness,elongation,fruit
0,-0.432637,-0.126039,1.0
1,1.148003,0.449690,1.0
2,1.765190,0.157238,1.0
3,-1.302076,0.591975,1.0
4,-1.604870,1.109809,1.0
...,...,...,...
486,1.494568,1.002116,-1.0
487,0.684247,1.643895,-1.0
488,0.188271,2.106111,-1.0
489,0.224988,3.117721,-1.0


<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Your final answers to question 1. </b> What are the mean and standard deviation of the lines and the columns of the `redness` and `elongation` features before and after standardization?

In [10]:
question = 1
score = 0
# Put your answers to question 1 here
redness_mean_before, elongation_mean_before, label_mean_before = df.mean()
redness_std_before, elongation_std_before, label_std_before = df.std()
redness_mean_after, elongation_mean_after, label_mean_after = df_scaled.mean() 
redness_std_after, elongation_std_after, label_std_after = df_scaled.std()

# This is the checker code, keep it
answer = redness_mean_before+elongation_mean_before+label_mean_before
answer += redness_std_before+elongation_std_before+label_std_before
answer -= redness_mean_after+elongation_mean_after+label_mean_after
answer -= redness_std_after+elongation_std_after+label_std_after
score += check(answer, question)

39.65876749424191
39.6587674942419


### Question 2: Write a function to standardize data
It is often useful to `standardize` the columns of the data matrix to put all values in a similar scale, so to facilitate re-using this operation, write a function that takes as input a data frame `df` containing a dataset and returns a dataframe `df_scaled` with the columns standardized <b>EXCEPT THE TARGET VALUES</b> of the last column (use your answers to the previous section). Use this template for your code:

    def standardize_df(df):
    '''Standardize all the columns except the last one (target values).'''
    # YOUR CODE HERE
    return df_scaled
    
Test your function with the same dataframe as in the previous questions and use the methods `head` and `describe` to verify that all the columns are standardized, except the last one. Notice that, due to machine precision, you may not get exactly mean=0 and std=1.

In [11]:
# Replace by your own code:
def standardize_df(df):
    '''Standardize all the columns except the last one (target values).'''
    df_scaled = df.copy()
    df_scaled.iloc[:,:-1] = (df_scaled.iloc[:,:-1] - df_scaled.iloc[:,:-1].mean()) / df_scaled.iloc[:,:-1].std()
    return df_scaled

df_scaled2 = standardize_df(df)
df_scaled2.head()

,redness,elongation,fruit
0,-0.432637,-0.126039,1.0
1,1.148003,0.449690,1.0
2,1.765190,0.157238,1.0
3,-1.302076,0.591975,1.0
4,-1.604870,1.109809,1.0


In [12]:
df_scaled2.describe()

,redness,elongation,fruit
count,4.910000e+02,4.910000e+02,491.000000
mean,-1.085350e-16,-8.266752e-16,0.356415
std,1.000000e+00,1.000000e+00,0.935280
min,-2.976736e+00,-8.998221e-01,-1.000000
25%,-6.405668e-01,-5.895726e-01,-1.000000
50%,-1.597070e-01,-3.238798e-01,1.000000
75%,7.330612e-01,2.479972e-01,1.000000
max,3.783535e+00,7.221221e+00,1.000000


In [13]:
# This is the checker code, keep it
question = 2
answer = (df_scaled == df_scaled2).all().all()
score += check(answer, question)

True
1


<span style="color:red"> 
    <b>You can now fill `answer__01(data)` in answer.py</b>
</span>


### Question 3: Select the best representation
We are now going to loop over all datasets and compare the performance of our baseline method (one nearest neighbor classifier) with and without variable scaling. 

First you will perform these steps (with some help):
* Import `KNeighborsClassifier` from the scikit-learn library (`sklearn`) and instanciate a one nearest neighbor classifier that you will call `sklearn_model`. Also import the `balanced_accuracy_score` and name it `sklearn_metric`.
* Call `df_cross_validate(df, sklearn_model, sklearn_metric)` and collect the results, then print the training and test performance and their error bars. <b>Tip:</b> look at the bottom of the code using `??df_cross_validate` to get an idea on how to print the results. 
* Create:
 * a list containing all the dataset dataframes and call it `all_data_df`
 * a list containg all the dataset names and call it `data_name`. 
* Run the function `systematic_data_experiment(data_name, all_scaled_data_df, sklearn_model, sklearn_metric)` and display the results.
* <b>Create a list</b> containing all the SCALED datasets (variables standardized) and call it `all_scaled_data_df`.
* Run again `systematic_data_experiment` on `all_scaled_data_df` and display the results.
* Fuse the results of the two previous question by creating a dataframe called `joint_results`. Assign `result_scaling.perf_te` to a column called `'SCALED'` and `result_noscaling.perf_te` to a column called `'NOT SCALED'`. Display the results.
* Visualize `joint_results` with a histogram. Check <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.bar.html">this documentation</a>.

Then the questions you should answer are:
* Does re-scaling variables always help?
* Is which case does it help most?

Try to reason why this might be.

<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Make necessary changes; USE THE DOCUMENTATION OF SCIKIT LEARN (do not copy on your neighbor).</b> 

In [14]:
from sklearn.naive_bayes import GaussianNB # Replace that by nearest neighbors
from sklearn.metrics import accuracy_score as sklearn_metric # Replace that by balanced accuracy 
sklearn_model = KNeighborsClassifier(n_neighbors=1) # Replace by ONE nearest neighbor

<span style="color:red"> 
    <b>You can now fill `answer__02()` in answer.py</b>
</span>


<span style="color:red"> 
    <b>You can now fill `answer__03()` in answer.py</b>
</span>


<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Collect the results of df_cross_validate(df, sklearn_model, sklearn_metric).</b> 

In [15]:
verbose = True
p_te, s_te, p_tr, s_tr  = 0,  0, 0, 0 # REPLACE THIS!

metric_name = sklearn_metric.__name__.upper()
df_cross = df_cross_validate(df, sklearn_model, metric_name)

print("AVERAGE TRAINING {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_tr, s_tr))
print("AVERAGE TEST {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_te, s_te))

TypeError: 'str' object is not callable

In [ ]:
# We give you this code, read it and try to understand it
data_name = [os.path.basename(file)[:-9] for file in data_list]
all_data_df = [pd.read_csv(file) for file in data_list]
print('NO SCALING')
result_noscaling = systematic_data_experiment(data_name, all_data_df, sklearn_model, sklearn_metric)
result_noscaling.style.background_gradient(cmap='Blues')

What do you observe by comparing the accuracy on training data and on test data?

<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Compute all_scaled_data_df in a one-line formula. Imitate the code in the previous cell.</b> 

In [ ]:
all_scaled_data_df = all_data_df # CHANGE THIS!

print('WITH SCALING')
result_scaling = systematic_data_experiment(data_name, all_scaled_data_df, sklearn_model, sklearn_metric)
result_scaling.style.background_gradient(cmap='Blues')

In [ ]:
# This is how to collect the test results in a single dataframe
joint_results = pd.DataFrame()
joint_results['SCALED'] = result_scaling.perf_te
joint_results['NOT SCALED'] = result_noscaling.perf_te
joint_results

<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Put here the code to plot "joint_results" as a bar graph. Check the <a href="Check https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.bar.html"> documentation</a></b>.

In [ ]:
joint_results.boxplot() # REPLACE THIS!
plt.ylabel(sklearn_metric.__name__)

<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Your final answers to question 3:</a></b> Does rescaling always help? In which case does it help most?

In [ ]:
# Put your answer to question 3 here
rescaling_always_help = 100 # 0 for false, 1 for true
varnum_rescaling_helps_most = -1 # num of var (0-based) for which perf. improve most after rescaling
case_num_rescaling_helps_most = -1 # num of case (0-based) for which perf. improve most after rescaling

# This is the checker code, keep it
question = 3
answer = (p_tr+s_tr+p_te+s_te)/10
answer += joint_results.sum().sum()/10-rescaling_always_help-case_num_rescaling_helps_most
score += check(answer, question)

<span style="color:red"> 
    <b>You can now fill `answer__04()` in answer.py</b>
</span>


<span style="color:red"> 
    <b>You can now fill `answer__05()` in answer.py</b>
</span>


## Step 2: Identify the best model
We are now ready to perform systematic experiments on various models. We will use the methods proposed in the variable `classifiers` of <a href="https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html">these scikit-learn examples</a>, compare classifier performances on the dataset `CS_SCALED`:

### Question 4: Overfitting and underfitting

</b> There may be two reasons why a model performs poorly. It could either be <b>overfitting</b> or <b>underfitting</b> data. 

First you will perform these steps (with some help):

* Create a variable `data_df` and assign to it the data frame of the scaled version of the CS dataset.
* Create a variable `model_name` and a variable `model_list` containing the list of model names and the list of models (classifiers) from the <a href="https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html">the scikit-learn examples</a> we pointed you to.
* Call `systematic_model_experiment(data_df, model_name, model_list, sklearn_metric)` and display the results. 
* Find which method performs best. <b>Tip:</b> Use <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.idxmax.html#pandas.Series.idxmax">idxmax</a>.
* Find which models have TEST performance UNDER THE MEDIAN test performance. The single out models with TRAINING performance UNDER THE MEDIAN and those with TRAINING performance OVER THE MEDIAN.

Then answer those questions:
* If the test performance is bad but the training performance is good, is the model under-fitting or over-fitting? 
* If both are bad, is the model is under-fitting or over-fitting? 
* Which models are over-fitted and which ones are under-fitted?

<span style="color:red"> 
    <b>You can now fill `answer__06()` in answer.py</b>
</span>


<span style="color:red"> 
    <b>You can now fill `answer__07()` in answer.py</b>
</span>


<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Choose the correct dataset: CS representation, scaled data.</b>

In [ ]:
dataset_choice = 0 # Change that
data_df = all_data_df[dataset_choice] # Change that

print(data_df.shape)

#### Classifier comparison
We import a bunch of classifiers, inspired 
by <a href="https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html">that list</a>.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model_name = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]
model_list = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=10),
    RandomForestClassifier(max_depth=10, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

#### Systematic experiments:
We give you this code to make systematic experiments (run all models on the chosen representation). You may get a warning. What could explain it?

In [ ]:
compar_results = systematic_model_experiment(data_df, model_name, model_list, sklearn_metric)
compar_results.round(2).style.background_gradient(cmap='Blues')

<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Find the method performing best on test data.</b>

In [ ]:
# Find the method performing best on test data
best_method = "AdaBoost" # REPLACE THIS; try not to just do this by hand, apply idxmax to the series perf_te
print("Best method: {}".format(best_method))

#### Over-fitting and under-fitting
There may be two reasons why a model performs poorly. It could either be <b>over-fitting</b> or <b>under-fitting</b> data. Under-fitting means that the model is not powerful enough to even learn the training data while over-fitting means that the model is so powerful that it can learn super-well the training data, but it might not generalize well to new test data.

Of the models performing poorly (having TEST performance UNDER THE MEDIAN test performance), we highlight:
* models with TRAINING performance UNDER THE MEDIAN training performance (<b>under-fitted</b>)
* models with TRAINING performance OVER THE MEDIAN (<b>over-fitted</b>)

In [ ]:
# We give you this code, check it in utilities.py of use ??analyze_model_experiments
analyze_model_experiments(compar_results)

#### Bar graph comparing  results

In [ ]:
compar_results[['perf_tr', 'perf_te']].plot.bar()
plt.ylim(0.5, 1)
plt.ylabel(sklearn_metric.__name__)

<div style="background:#FFFFAA">
    Here you need to do something!
</div>
<b>Your final answers to question 4:</a> </b>Which methods overfits or underfit?</b>

In [ ]:
# Put your answer to question 4 here
#If the test performance is bad but the training performance is good, is the model under-fitting or over-fitting?
answer1 = -1 # 0 for under-fitting and 1 for over-fitting
#If both are bad, is the model is under-fitting or over-fitting? 
answer2 = -1 # 0 for under-fitting and 1 for over-fitting
# Which models are over-fitted and which ones are under-fitted?
overfitted_list = [1,3,6] # Replace by the correct numbers in model_name
underfitted_list = [2,8] # Replace by the correct numbers in model_name

model_name = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

print("Over_fitted models:")
print([model_name[i] for i in overfitted_list])
print("Under_fitted models:")
print([model_name[i] for i in underfitted_list])
# This is the checker code, keep it
question = 4
answer = answer1-answer2+sum(overfitted_list)-sum(underfitted_list)+model_name.index(best_method)

score += check(answer, question)

<span style="color:red"> 
    <b>You can now fill `answer__08()` in answer.py</b>
</span>


<span style="color:red"> 
    <b>You can now fill `answer__09()` in answer.py</b>
</span>


###  Question 5: Dimensionality reduction
It is useful to reduce the data dimension (number of features) for two reasons: ease of visualisation and possibly increase in performance. Two approaches are possible:
1. Feature selection.
1. Feature transforms.

In the first case, one tries to select among the original features. In the second case, one first replaces the original features by "combinations" of features, then perform selection. Here we give two simple examples: Feature ranking with the <b>Pearson correlation</b> coefficient (as a feature selection method), and <b>Singular Value Decomposition</b> or SVD (as a feature transform method) .

In this section, we keep using the <b>CS scaled data</b>. We use as classifier the <b>3-nearest-neighbor</b> model, which seems to have served us well! We begin by showing you how to do simple feature selection, then we'll guide you step-by-step through SVD.

### Feature selection example
We perform feature selection with the Pearson correlation coefficient as follows:
* Compute the correlation matrix (we will only use the last column, i.e. correlation of features with the class label to be predicted)
* Sort the correlation coefficients (of features and class label)

#### Correlation matrix
If we did things right, `data_df` should contain the CS data. 

In [ ]:
corr = data_df.corr()
corr.round(1).style.background_gradient(cmap='coolwarm')

#### Sort features

Let us first sort all features by the <b>absolute value</b> of the Pearson correlation coefficient. Indeed, variables are informative no matter whether they are correlated or anti-correlated (since it suffices to multiply them by -1 to change the correlation direction).

In [ ]:
sval = corr['fruit'][:-1].abs().sort_values(ascending=False)
ranked_columns = sval.index.values
print(ranked_columns) 

We notice that the features that we have constructed in the previous lessons 'R-(G+B)/2' and 'W/H' come in the 5 top most informative features. But there are others. Let us make all scatter plots of pairs of features for the 5 top ranked features.

#### Visualize top 5 features with PAIRPLOT

In [ ]:
# Make a dataframe with only top five features
fruit_name = ['Banana', 'Apple']
fruit_list = [fruit_name[int((i+1)/2)] for i in data_df["fruit"].tolist()]
col_selected = ranked_columns[0:5]
df_5feat = pd.DataFrame.copy(data_df)
df_5feat = df_5feat[col_selected]
df_5feat['fruit'] = fruit_list
df_5feat.head()

In [ ]:
# Show pairplot
g = sns.pairplot(df_5feat, hue="fruit", markers=["o", "s"], diag_kind="hist")

#### Learning curves
We want to eveluate the effect of varying the number of features. To that end, we build learning curves = performance as a function of feature number.

In [ ]:
# We use the 3 nearest neighbor classifier to create the learnign curve
sklearn_model = KNeighborsClassifier(n_neighbors=3)
feat_lc_df = feature_learning_curve(data_df, sklearn_model, sklearn_metric)

In [ ]:
# Plot the learning curve
plt.errorbar(feat_lc_df.index+1, feat_lc_df['perf_tr'], yerr=feat_lc_df['std_tr'], label='Training set')
plt.errorbar(feat_lc_df.index+1, feat_lc_df['perf_te'], yerr=feat_lc_df['std_te'], label='Test set')
plt.xticks(np.arange(1, 22, 1)) 
plt.xlabel('Number of features')
plt.ylabel(sklearn_metric.__name__)
plt.legend(loc='lower right')

We see the, with 5 features, it is about as good as it gets, given the error bars.

### Singular Value Decomposition
Let us move now to the second way of reducing dimensionality: feature transforms.
When we use SVD, the feature transform consists in finding the "principal directions"(directions of largest variance). The method is also known as Pricipal Component Analysis (PCA). I simply used my search engine and typed the keywords "pandas svd". I found a nice tutorial on <a href="https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/">this page</a> and a step-by-step procedure on <a href="https://cmdlinetips.com/2019/05/singular-value-decomposition-svd-in-python/">this page</a>. 

We will guide you step-by-step:
* Create a dataframe called `df_scaled` containing the standardized columns, except the last one (tip: just use `drop` to eliminate the last column).
* Perform a singular value decomposition of `df_scaled` and call the resulting matrices u, s, v.
* Make a scree plot of the eigen values (square of the singular values). Save the plot in file 'svd_scree_plot.png'.
* Create a new dataframe `svd_df` with the two singular values as columns and the fruit type as index.
* Make pairwise scatter plots of the three first singular values.

Then the question to answer will be to compute the performances obtained with the 3 nearest neighbor method using the first 3 singular values.

<div style="background:#FFFFAA">
    Here you need to do something!
</div>

In [ ]:
# Make use the CS scaled dataset (this should be already loaded in data_df,
# if you did things correctly in previous questions)
# To obtain df_scaled, remove the last column of data_df.

df_scaled = df_scaled2[{"redness", "elongation"}] # REPLACE THIS BY THE CORRECT ANSWER

df_scaled.head()

#### Perform singular value decomposition

In [ ]:
u, s, v = np.linalg.svd(df_scaled, full_matrices=True)
print('U {}'.format(u.shape))
print('S {}'.format(s.shape))
print('V {}'.format(v.shape))

#### Make a scree plot
This plot allows us to decide how many components to keep, considering the total variance explained.

In [ ]:
var_explained = np.round(s**2/np.sum(s**2), decimals=3)
sns.barplot(x=list(range(1,len(var_explained)+1)),
            y=var_explained, color="limegreen")
plt.xlabel('SVs', fontsize=16)
plt.ylabel('Percent Variance Explained', fontsize=16)
plt.savefig('svd_scree_plot.png',dpi=100)

We see that after 3 features there is a big drop in variance. So tentatively, let us keep the top three components.

#### Pairplot

In [ ]:
# Create a dataframe with only the three first principal components
fruit_name = ['Banana', 'Apple']
fruit_list = [fruit_name[int((i+1)/2)] for i in df["fruit"].tolist()]
fnum=3
labels= ['SV'+str(i) for i in range(1,fnum+1)]
df_3svd = pd.DataFrame(u[:,0:fnum], columns=labels)
df_3svd['fruit'] = fruit_list
df_3svd.head()

<div style="background:#FFFFAA">
    Here you need to do something!
</div>

In [ ]:
# Show the pairplot of svd_df

#### Evaluate df_3svd: 
This piece of code will serve you later to answer question 5!

In [ ]:
# We use the function df_cross_validate that we already used in question 3 to evaluate svd_df
df_3svd['fruit'] = df.iloc[:, -1].to_numpy() # We replace the target values by numbers
p_tr, s_tr, p_te, s_te = df_cross_validate(df_3svd, sklearn_model, sklearn_metric)
metric_name = sklearn_metric.__name__.upper()
print("AVERAGE TRAINING {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_tr, s_tr))
print("AVERAGE TEST {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_te, s_te))

#### SVD learning curve:

In [ ]:
# We compute the learning curve for the 3-nearest neighbor classifier ...
sklearn_model = KNeighborsClassifier(n_neighbors=3)
feat_lc_df = svd_learning_curve(data_df, sklearn_model, sklearn_metric)

# and we plot the learning curve
plt.errorbar(feat_lc_df.index+1, feat_lc_df['perf_tr'], yerr=feat_lc_df['std_tr'], label='Training set')
plt.errorbar(feat_lc_df.index+1, feat_lc_df['perf_te'], yerr=feat_lc_df['std_te'], label='Test set')
plt.xticks(np.arange(1, 22, 1)) 
plt.xlabel('Number of principal compoments')
plt.ylabel(sklearn_metric.__name__)
plt.legend(loc='lower right')

#### Comparison of dimensionality reduction methods 

Even though features 4 and 5 did not seem to explain a lot of additional variance, there seems to be an optimum at 4 or 5 features. Compare the performances of df_5feat and df_5svd (with 5 top components). Is one significantly better then the other?

<div style="background:#FFFFAA">
    Here you need to do something!
</div>

In [ ]:
df_5svd = df_3svd # Replace with the correct dataframe having 5 principal components

# Put here the code to compute the performances of df_5feat and df_5svd
fnum=5
labels= ['SV'+str(i) for i in range(1,fnum+1)]
df_5svd = pd.DataFrame(u[:,0:fnum], columns=labels)
df_5svd['fruit'] = df.iloc[:, -1].to_numpy() # Need numeric values
p_tr_5svd, s_tr_5svd, p_te_5svd, s_te_5svd = 0,0,0,0 # REPLACE THIS
p_tr_5feat, s_tr_5feat, p_te_5feat, s_te_5feat = 0,0,0,0 # REPLACE THIS

print("5FEAT AVERAGE TRAINING {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_tr_5feat, s_tr_5feat))
print("5FEAT AVERAGE TEST {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_te_5feat, s_te_5feat))
print("5SVD AVERAGE TRAINING {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_tr_5svd, s_tr_5svd))
print("5SVD AVERAGE TEST {0:s} +- STD: {1:.2f} +- {2:.2f}".format(metric_name, p_te_5svd, s_te_5svd))

In [ ]:
#Does one method give significantly better results than the other?
answer = -1 # One for yes and 0 for no

# This is the checker code, keep it
question = 5
answer = 0.001*(p_te_5feat-p_te_5svd)/(s_te_5feat+s_te_5svd)
score += check(answer, question)

In [ ]:
print('Your final score is %d / 5, congratulations!' % score)

<div style="background:#FFFFAA">
<span style="color:red">
<br>
    To finalize your homework:
<b>
<ul>
    <li> Use  Kernel + Restart and Run all.</li>
    <li> Save your notebook.</li>
    <li> Push your changes to your GitHub repo with:</li>
</ul>   
</b>
<pre>
git add .
git commit -m 'my homework is done'
git push
</pre>
<br>
</span>
</div>